In [ ]:
import warnings
warnings.filterwarnings("ignore", category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)
#Libraries for Data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import duckdb

In [ ]:
#load csvs
airportComments = pd.read_csv('airport-comments.csv')
airportFrequencies = pd.read_csv('airport-frequencies.csv')
airports = pd.read_csv('airports.csv')
countries = pd.read_csv('countries.csv')
navaids = pd.read_csv('navaids.csv')
regions = pd.read_csv('regions.csv')
runways = pd.read_csv('runways.csv')

In [ ]:
#Figure out types of data and null values
airportComments.info()
airportComments.isnull().sum()

In [ ]:
airportFrequencies.info()
airportFrequencies.isnull().sum()

In [ ]:
airports.info()
airports.isnull().sum()

In [ ]:
countries.info()
countries.isnull().sum()

In [ ]:
navaids.info()
navaids.isnull().sum()

In [ ]:
regions.info()
regions.isnull().sum()

In [ ]:
runways.info()
runways.isnull().sum()

In [ ]:
#Clean up data
airportComments.columns = airportComments.columns.str.strip().str.replace('"', '')
airportComments.dropna(subset = ['threadRef'], inplace = True)
airportComments[['memberNickname', 'subject', 'body']] = airportComments[['memberNickname', 'subject', 'body']].fillna("Blank")
airportComments['date'] = pd.to_datetime(airportComments['date'])

airportFrequencies['description'].fillna("Blank", inplace = True)

airports.drop(['icao_code', 'iata_code', 'gps_code', 'local_code', 'home_link', 'wikipedia_link', 'keywords'], axis=1, inplace = True)
airports[['municipality', 'iso_country', 'continent']] = airports[['municipality', 'iso_country', 'continent']].fillna("Blank")
airports['elevation_ft'] = airports['elevation_ft'].interpolate()

countries[['code', 'continent', 'keywords']] = countries[['code', 'continent', 'keywords']].fillna("Blank")

navaids.drop(['dme_frequency_khz','dme_channel','dme_latitude_deg', 'dme_longitude_deg','dme_elevation_ft','slaved_variation_deg'],
             axis = 1, inplace = True)
navaids[['ident', 'iso_country', 'usageType', 'power', 'associated_airport']] = navaids[['ident', 'iso_country', 'usageType', 'power', 'associated_airport']].fillna("Blank")
navaids[['elevation_ft', 'magnetic_variation_deg']] = navaids[['elevation_ft', 'magnetic_variation_deg']].interpolate()

regions[['local_code', 'continent', 'iso_country', 'wikipedia_link', 'keywords']] = regions[['local_code', 'continent', 'iso_country', 'wikipedia_link', 'keywords']].fillna("Blank")

runways = runways[['id', 'airport_ref', 'airport_ident', 'length_ft', 'width_ft', 'surface', 'lighted', 'closed']]
runways['surface'].fillna("Blank", inplace = True)
runways[['length_ft', 'width_ft']] = runways[['length_ft', 'width_ft']].interpolate()

In [ ]:
#Check to see if df's are all clean
airportComments.isnull().sum()
airportComments.info()

In [ ]:
airportFrequencies.isnull().sum()
airportFrequencies.head(10)

In [ ]:
airports.isnull().sum()

In [ ]:
countries.isnull().sum()

In [ ]:
navaids.isnull().sum()

In [ ]:
regions.isnull().sum()
regions.head()

In [ ]:
runways.isnull().sum()

In [ ]:
#Count runways per airport
airport_features = airports[['id', 'ident', 'type', 'name', 'elevation_ft', 'latitude_deg', 'longitude_deg']].copy()

runway_counts = runways.groupby('airport_ident').agg({
    'id': 'count',
    'length_ft': 'mean',
    'width_ft': 'mean',
    'lighted': 'sum'
}).rename(columns={'id': 'runway_count', 'length_ft': 'avg_runway_length', 
                   'width_ft': 'avg_runway_width', 'lighted': 'lighted_runways'})

runway_counts.head()

In [ ]:
# Count navaids per airport
navaid_counts = navaids.groupby('associated_airport').agg({
    'id': 'count',
    'type': lambda x: x.nunique()  # navaid diversity
}).rename(columns={'id': 'navaid_count', 'type': 'navaid_types'})

# Count frequencies per airport
freq_counts = airportFrequencies.groupby('airport_ident').agg({
    'id': 'count',
    'type': lambda x: x.nunique()
}).rename(columns={'id': 'frequency_count', 'type': 'frequency_types'})

In [ ]:
# Merge all together
airport_features = airport_features.merge(runway_counts, left_on='ident', right_index=True, how='left')
airport_features = airport_features.merge(navaid_counts, left_on='ident', right_index=True, how='left')
airport_features = airport_features.merge(freq_counts, left_on='ident', right_index=True, how='left')

In [ ]:
#Drop null values from new feature df
airport_features.dropna(inplace = True)

In [ ]:
#Drop non-feature columns
X = airport_features.drop(columns = ['id', 'ident', 'type', 'name'])

In [ ]:
#Standardize
col = X.columns
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X))
X.columns = col
X

In [ ]:
#Best n_components found was 2, cov of tied, and pca of 2
pca = PCA(n_components = 2)
X = pca.fit_transform(X)
gmm = GaussianMixture(n_components = 2, covariance_type = 'tied', random_state = 42)
gmm_labels = gmm.fit_predict(X)
gmm_silhouette = silhouette_score(X, gmm_labels)

In [ ]:
#Plot the original data
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.scatter(X[:, 0], X[:, 1], c='k', s=25)
plt.xlabel('1st Principle Component')
plt.ylabel('2nd Principle Component')
plt.title("Original Data")

#Plot the GMM clustering results
plt.subplot(1, 3, 2)
plt.scatter(X[:, 0], X[:, 1], c=gmm_labels, cmap='viridis', s=25)
plt.title("GMM Clustering")
plt.xlabel('1st Principle Component')
plt.ylabel('2nd Principle Component')
plt.title(f"GMM Clustering\nSilhouette Score: {gmm_silhouette:.2f}")

plt.show()

In [ ]:
#Temporal comment analysis
query = """
WITH TotalComments AS(
    SELECT
        ac.date as date,
        a.name AS airport_name,
        a.id AS airport_id,
        ac.id AS comment_id
    FROM
        airports AS a
    JOIN
        airportComments AS ac ON a.ident = ac.airportIdent
)
SELECT
    EXTRACT(YEAR from date) as year,
    EXTRACT(MONTH FROM date) as month,
    tc.airport_name,
    COUNT(tc.comment_id) AS total_comments
FROM
    TotalComments AS tc
GROUP BY
    tc.airport_name, year, month
ORDER BY
    total_comments DESC;
"""
result = duckdb.query(query).df()
result

In [ ]:
result['date'] = pd.to_datetime(result[['year', 'month']].assign(day=1))

In [ ]:
#graph total comments against time and label each point with airport name
plt.figure(figsize=(18,5))
plt.scatter(result['date'], result['total_comments'], c = 'blue')
plt.yscale('log')
plt.xlabel('Date')
plt.ylabel('Total Comments')
plt.title('Total Comments vs Date for Airports')
plt.show()

In [ ]:
#same but with plotly
import plotly.express as px
fig = px.scatter(
    result, 
    x=result['date'], 
    y=result['total_comments'],  
    hover_name=result['airport_name'] 
)

fig.update_layout(
    title='Total Comments vs Date for Airports',
    xaxis_title='Date',
    yaxis_title='Total Comments'
)

fig.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have X (features) and y (target)
# X can be a single column or multiple columns
# y is what you're trying to predict

X = result.drop(columns = ['total_comments', 'date']) 
X = pd.get_dummies(X)
col = X.columns
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns = col)
y = result['total_comments']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on TEST set for evaluation
y_pred = model.predict(X_test)

# Create scatter plot
fig = px.scatter(result, 
    x='date', 
    y='total_comments',  
    hover_name='airport_name' 
)

# Add regression line using ALL predictions
fig.add_trace(
    go.Scatter(
        x=result['date'],
        y=y_pred,  
        mode='lines',
        name='Linear Regression',
        line=dict(color='red', width=2)
    )
)

fig.update_layout(
    title='Total Comments vs Date for Airports',
    xaxis_title='Date',
    yaxis_title='Total Comments'
)
fig.show()

# Print regression stats (on test set)
print(f"Test R² Score: {r2_score(y_test, y_pred)}")
print(f"Test MSE: {mean_squared_error(y_test, y_pred)}")
print(f"Train R² Score: {model.score(X_train, y_train)}")


In [ ]:
# Run query
query = """
SELECT
    a.name AS airport_name,
    a.id AS airport_id,
    a.type AS airport_type,
    c.name AS country_name,
    r.name AS region_name,
    ROUND(AVG(f.frequency_mhz),3) AS average_frequency_mhz,
    COUNT(*) as count
FROM airports AS a
    JOIN airportFrequencies AS f ON a.ident = f.airport_ident
    JOIN regions AS r ON a.iso_region = r.code
    JOIN countries AS c ON c.code = r.iso_country
GROUP BY a.name, a.id, a.type, c.name, r.name
ORDER BY a.type DESC, average_frequency_mhz DESC;
"""

df = duckdb.query(query).df()

In [ ]:
# Remove top and bottom 5% of frequencies
lower_percentile = df['average_frequency_mhz'].quantile(0.01)
upper_percentile = df['average_frequency_mhz'].quantile(0.99)
df_filtered = df[(df['average_frequency_mhz'] >= lower_percentile) & 
                  (df['average_frequency_mhz'] <= upper_percentile)]

In [ ]:
df[df['average_frequency_mhz'] >= upper_percentile].head(20)

In [ ]:
#sunburst diagram
fig = px.sunburst(
    df_filtered,
    path=['country_name', 'region_name', 'airport_type'],
    values='count',
    color='average_frequency_mhz',
    color_continuous_scale='Plasma',
    title='Hierarchical Distribution of Airport Communication Frequencies',
    width = 1000,
    height = 800
)

fig.update_traces(textinfo="label+percent parent+value")
fig.show()

In [ ]:
query = """
SELECT
    a.iso_country,
    a.iso_region,
    a.type,
    a.elevation_ft as elv,
    COUNT(*) as count
FROM airports AS a
GROUP BY a.iso_country, a.iso_region, a.type, elv
ORDER BY count DESC;
"""

df = duckdb.query(query).df()

In [ ]:
#alt sunburst
fig = px.sunburst(
    df,
    path=['iso_country', 'iso_region', 'type'],
    values='count',
    color='elv',
    color_continuous_scale='Plasma',
    title='Hierarchical Distribution of Airports by Country, Region, and Type with Elevation Heatmap',
    width = 1000,
    height = 800
)

fig.update_traces(textinfo="label+percent parent+value")
fig.show()

In [ ]:
df = airportFrequencies[['type', 'frequency_mhz']]
target = np.array(df['type'].value_counts().head(10).index)
mask = df['type'].isin(target)
df = df[mask]

In [ ]:
df = airports[['iso_country', 'elevation_ft']]
target = np.array(df['iso_country'].value_counts().head(10).index)
mask = df['iso_country'].isin(target)
df = df[mask]

In [ ]:
#violin plot of frequencies
fig = px.violin(
    df, 
    x='iso_country', 
    y='elevation_ft', 
    color='iso_country',
    box=True,  # Optionally add a box plot inside the violin
    points='all', # Optionally show all data points
    title="Violin Plot of Elevation (ft) by Top 10 Countries with the Most Airports",
    width=1000,
    height=800
)

fig.show()